In [79]:
import json
import pandas as pd

train_list = []
file_name = "train_time.txt"
with open(file_name, "r", encoding="utf-8") as f:
    for line in f:
        if line[0] in ["G", "D", "C", "S", "Z", "Y", "T", "K", "N"]:
            train_list.append({"train_id": "", "train_time": []})
            train_list[-1]["train_id"] = line.strip()
        elif line == "" or line == "\n":
            continue
        else:
            train_list[-1]["train_time"].append(line.strip().split("\t"))

with open("train_time.json", "w", encoding="utf-8") as f:
    json.dump(train_list, f, ensure_ascii=False, indent=4)

In [80]:
s = set()
for train in train_list:
    for stop in train["train_time"]:
        s.add(stop[0])
total = len(s)

df = pd.DataFrame(s).sort_values(by=0).reset_index(drop=True)
df.columns = ["station"]
df["station_id"] = range(0, total)
d={}
for i in range(total):
    d[df["station"][i]] = str(df["station_id"][i])
with open("station_id.json", "w", encoding="utf-8") as f:
    json.dump(d, f, ensure_ascii=False, indent=4)

In [81]:
station2id = {}
for id in range(0, total):
    station2id[df.iloc[id, 0]] = id

for train in train_list:
    for stop in train["train_time"]:
        stop.append(station2id[stop[0]])

with open("train_time.json", "w", encoding="utf-8") as f:
    json.dump(train_list, f, ensure_ascii=False, indent=4)

In [82]:
d

{'一步滩': '0',
 '一面坡': '1',
 '一面坡北': '2',
 '七台河': '3',
 '七台河西': '4',
 '七甸': '5',
 '七苏木': '6',
 '七营': '7',
 '七龙星': '8',
 '万乐': '9',
 '万发屯': '10',
 '万宁': '11',
 '万安县': '12',
 '万州': '13',
 '万州北': '14',
 '万年': '15',
 '万源': '16',
 '万荣': '17',
 '万象': '18',
 '三义井': '19',
 '三亚': '20',
 '三亚2': '21',
 '三元坝': '22',
 '三元西': '23',
 '三关口': '24',
 '三原': '25',
 '三合庄': '26',
 '三家子': '27',
 '三家寨': '28',
 '三明': '29',
 '三明北': '30',
 '三星堆': '31',
 '三水北': '32',
 '三水南': '33',
 '三汇镇': '34',
 '三江': '35',
 '三江南': '36',
 '三江口': '37',
 '三河县': '38',
 '三源浦': '39',
 '三灶东': '40',
 '三穗': '41',
 '三营': '42',
 '三道营': '43',
 '三都县': '44',
 '三门县': '45',
 '三门峡': '46',
 '三门峡南': '47',
 '三门峡西': '48',
 '三间房': '49',
 '三阳': '50',
 '三阳川': '51',
 '上万': '52',
 '上仓': '53',
 '上普雄': '54',
 '上杭': '55',
 '上板城': '56',
 '上板城南': '57',
 '上海': '58',
 '上海南': '59',
 '上海虹桥': '60',
 '上海西': '61',
 '上腰墩': '62',
 '上虞': '63',
 '上虞南': '64',
 '上西铺': '65',
 '上谷': '66',
 '上饶': '67',
 '上高镇': '68',
 '下仓': '69',
 '下台子': '70',
 '下城子': '71',
 '下普雄': '72',
 '下板城'

In [83]:
import datetime, math

INTERVAL = 15
SPLIT = 24 * 60 // INTERVAL
STATION_END = total * SPLIT * 2
LONGEST_TIME = 6 * 60 // INTERVAL


def station_in(station_id):
    return station_id * SPLIT * 2


def station_out(station_id):
    return station_id * SPLIT * 2 + SPLIT


def transfer(station_id, time, io):
    time = datetime.datetime.strptime(time, "%H:%M")
    if io == 0:
        time = math.ceil((time.hour * 60 + time.minute) / 15) % SPLIT
    else:
        time = math.floor((time.hour * 60 + time.minute) / 15) % SPLIT
    return int(station_id * SPLIT * 2 + time + io * SPLIT)


file = open("in.txt", "w")
file.write(str(len(d.keys())) + "\n")
for i, v in d.items():
    file.write(str(i) + " " + str(v) + "\n")
# file.write(str(STATION_END + total)+"\n")
for train in train_list:
    for f in range(0, len(train["train_time"]) - 1):
        for t in range(f + 1, len(train["train_time"])):
            starttime = datetime.datetime.strptime(train["train_time"][f][2], "%H:%M")
            endtime = datetime.datetime.strptime(train["train_time"][t][1], "%H:%M")
            if (endtime - starttime).seconds / 60 < 0:
                endtime += datetime.timedelta(days=1)
            during = int((endtime - starttime).seconds / 60)
            f_station = transfer(
                train["train_time"][f][3], train["train_time"][f][2], 1
            )
            t_station = transfer(
                train["train_time"][t][3], train["train_time"][t][1], 0
            )
            file.write(
                str(f_station)
                + " "
                + str(t_station)
                + " "
                + str(during)
                + " "
                + train["train_id"]
                + "："
                + train["train_time"][f][0]
                + "至"
                + train["train_time"][t][0]
                + "从"
                + train["train_time"][f][2]
                + "到"
                + train["train_time"][t][1]
                + "\n"
            )

for i in range(total):
    for j in range(station_in(i), station_out(i)):
        for k in range(j + 1, j + LONGEST_TIME):
            if k + SPLIT < (i + 1) * SPLIT * 2:
                file.write(
                    str(j)
                    + " "
                    + str(k + SPLIT)
                    + " "
                    + str((k - j) * INTERVAL)
                    + " wait\n"
                )
            else:
                file.write(
                    str(j) + " " + str(k) + " " + str((k - j) * INTERVAL) + " wait\n"
                )

# for i in range(total):
#     for j in range(station_in(i), station_out(i)):
#         file.write(str(j) + " " + str(STATION_END + i) + " 0 end\n")

with open("city_list.txt", "r") as f:
    for line in f:
        city = line.split(" ")
        if city[0] in d and city[1] in d:
            fr, to = station_in(int(d[city[0]])), station_in(int(d[city[1]]))
            for i in range(fr, fr + SPLIT):
                for j in range(
                    to + math.ceil(int(city[2]) / INTERVAL), to + LONGEST_TIME
                ):
                    if j + SPLIT < (int(d[city[1]]) + 1) * SPLIT * 2:
                        file.write(
                            str(i)
                            + " "
                            + str(j + SPLIT)
                            + " "
                            + str((j - to) * INTERVAL)
                            + " T\n"
                        )
                    else:
                        file.write(
                            str(i)
                            + " "
                            + str(j)
                            + " "
                            + str((j - to) * INTERVAL)
                            + " T\n"
                        )
            fr, to = station_in(int(d[city[1]])), station_in(int(d[city[0]]))
            for i in range(fr, fr + SPLIT):
                for j in range(
                    to + i - fr + math.ceil(int(city[2]) / INTERVAL),
                    to + i - fr + +LONGEST_TIME,
                ):
                    # if fr == station_in(int(d['北京西'])):
                    #     print(i, to + math.ceil(int(city[2]) / INTERVAL) + SPLIT, to + LONGEST_TIME + SPLIT, station_out(int(d['北京'])))
                    if j + SPLIT < (int(d[city[0]]) + 1) * SPLIT * 2:
                        file.write(
                            str(i)
                            + " "
                            + str(j + SPLIT)
                            + " "
                            + str((j - (to + i - fr)) * INTERVAL)
                            + " T\n"
                        )
                    else:
                        file.write(
                            str(i)
                            + " "
                            + str(j)
                            + " "
                            + str((j - (to + i - fr)) * INTERVAL)
                            + " T\n"
                        )